In [76]:
import pandas as pd
from morfeusz2 import Morfeusz
morfeusz = Morfeusz() 

In [2]:
pd.set_option('display.max_colwidth', None)

## Loading data and choosing only appropiate MWE

In [77]:
poleval_df = pd.read_csv("poleval2019_task2_training_190221/index.tsv", sep="\t", header=None)
poleval_df.columns = ["phrase_id", "doc_id", "text", "lemma"]
poleval_df.head()

,phrase_id,doc_id,text,lemma
0,31822,99883,Toronto Dominion Centre,Toronto Dominion Centre
1,40025,99883,Toronto,Toronto
2,343873,99883,kompleks handlowo-kulturalny,kompleks handlowo-kulturalny
3,31833,99883,Joe Fafard,Joe Fafard
4,327365,99883,kanadyjskim,kanadyjski


In [78]:
print(f"Shape of the original Poleval 2019 dataset: {poleval_df.shape}")

Shape of the original Poleval 2019 dataset: (22177, 4)


In [79]:
mwe_df = poleval_df[poleval_df["text"].str.split().str.len() > 1]
mwe_df.head()

,phrase_id,doc_id,text,lemma
0,31822,99883,Toronto Dominion Centre,Toronto Dominion Centre
2,343873,99883,kompleks handlowo-kulturalny,kompleks handlowo-kulturalny
3,31833,99883,Joe Fafard,Joe Fafard
5,343872,99883,Ludwiga Mies van der Rohe,Ludwig Mies van der Rohe
7,343871,99883,Toronto Dominion Gallery of Inuit Art,Toronto Dominion Gallery of Inuit Art


In [80]:
print(f"Shape of the Poleval 2019 dataset (only multi word expressions): {mwe_df.shape}")

Shape of the Poleval 2019 dataset (only multi word expressions): (9718, 4)


**remove not polish MWE**


In [81]:
def is_polish_mwe(text):
    """
    Check if the text is a valid Polish MWE.
    Returns True if at least one word is recognized (not tagged as 'ign').
    """
    analysis = morfeusz.analyse(text)
    for _, _, interpretations in analysis:
        tag = interpretations[2]
        if tag != 'ign':
            return True
    return False

In [82]:
df_unrecognized = mwe_df[~mwe_df['text'].apply(is_polish_mwe)].reset_index(drop=True)
print(f"Number of unrecognized MWEs: {len(df_unrecognized)}")
df_unrecognized.sample(10)

Number of unrecognized MWEs: 566


,phrase_id,doc_id,text,lemma
558,328976,107357,Aldin Didić,Aldin Didić
518,486132,107260,José Luis Zapatero,José Luis Zapatero
557,327280,107351,Enduring Freedom,Enduring Freedom
234,72190,101768,Mitta Romneya,Mitt Romney
334,78132,102132,Philip Brickman,Philip Brickman
353,283886,102291,Deep Blue,Deep Blue
509,317672,107247,Tavistock Square,Tavistock Square
341,473773,102249,CEO Suna,CEO Suna
292,310392,102039,Wealth of networks,Wealth of networks
424,344212,103691,Wikimedia Commons,Wikimedia Commons


In [83]:
mwe_df = mwe_df[mwe_df["text"].apply(is_polish_mwe)].reset_index(drop=True)
print(f"Shape of the Poleval 2019 dataset (only valid Polish MWEs): {mwe_df.shape}")

Shape of the Poleval 2019 dataset (only valid Polish MWEs): (9152, 4)


There was 566 fully not recognized expressions by Morfeusz.


**Remove duplicates**


In [84]:
def remove_duplicates(df):
    """
    Remove duplicates from the DataFrame based on 'text' and 'lemma'.
    Keeps the first occurrence.
    """
    return df.drop_duplicates(subset=["text", "lemma"], keep='first')

In [85]:
mwe_df = remove_duplicates(mwe_df)
print(f"Shape of the Poleval 2019 dataset (after removing duplicates): {mwe_df.shape}")

Shape of the Poleval 2019 dataset (after removing duplicates): (7453, 4)


**TO DO: podzielić na nieodmienne i mianowniki:**


In [86]:
mwe_df[mwe_df["text"] == mwe_df["lemma"]]


,phrase_id,doc_id,text,lemma
0,31822,99883,Toronto Dominion Centre,Toronto Dominion Centre
1,343873,99883,kompleks handlowo-kulturalny,kompleks handlowo-kulturalny
2,31833,99883,Joe Fafard,Joe Fafard
4,343871,99883,Toronto Dominion Gallery of Inuit Art,Toronto Dominion Gallery of Inuit Art
5,343875,100499,PZL.13 (PZL-13),PZL.13 (PZL-13)
...,...,...,...,...
9137,329445,107360,Marin Leovac,Marin Leovac
9139,329446,107360,Roland Linz,Roland Linz
9142,329448,107360,Tomas Jun,Tomas Jun
9144,329449,107360,Patrick Salomon,Patrick Salomon


In [87]:
mwe_df = mwe_df[mwe_df["text"] != mwe_df["lemma"]]


In [88]:
print(f"Shape of the Poleval 2019 dataset (only inflected mwe): {mwe_df.shape}")


Shape of the Poleval 2019 dataset (only inflected mwe): (3828, 4)


In [89]:
mwe_df[['text', 'lemma']].sample(10)

,text,lemma
3466,Polski wschodniej,Polska wschodnia
6401,Kościele św. Józefa,Kościół św. Józefa
7416,Portalu mam.media.pl,Portal mam.media.pl
734,Lublin R-VIII a,Lublin R-VIII
5205,Samorządowego Kolegium Odwoławczego w Białymstoku,Samorządowe Kolegium Odwoławcze w Białymstoku
3632,festiwali Sputnik,festiwal Sputnik
8181,Cyfrowych Archiwów Tradycji Lokalnej,Cyfrowe Archiwa Tradycji Lokalnej
5403,Traktatu ustanawiającego Konstytucję dla Europy,Traktat ustanawiający Konstytucję dla Europy
211,Europejskiego Festiwalu Młodzieży,Europejski Festiwal Młodzieży
6675,Adama Kacperowskiego,Adam Kacperowski


## Cleaning

In [90]:
mwe_df.isna().sum()

phrase_id    0
doc_id       0
text         0
lemma        1
dtype: int64

In [91]:
mwe_df[mwe_df["lemma"].isna()]

,phrase_id,doc_id,text,lemma
1334,307628,101286,San Diego Rockets,NaN


In [92]:
# I drop it because it's not a polish MWE
mwe_df = mwe_df.dropna()

In [93]:
mwe_df.isna().sum()

phrase_id    0
doc_id       0
text         0
lemma        0
dtype: int64

In [94]:
mwe_df[mwe_df["text"] == "Dolce & Gabbana"]

,phrase_id,doc_id,text,lemma
161,99725,100524,Dolce & Gabbana,Dolce &amp; Gabbana


In [95]:
mwe_df[mwe_df["text"] == "Lublin R-VIII a"]

,phrase_id,doc_id,text,lemma
734,65875,101200,Lublin R-VIII a,Lublin R-VIII


In [96]:
mwe_df[mwe_df["text"] == "BBC Radio 1's Live Lounge"]

,phrase_id,doc_id,text,lemma
2264,426217,101434,BBC Radio 1's Live Lounge,BBC Radio 1


In [97]:
mwe_df[mwe_df["text"] == "Jammin' CRT.5"]

,phrase_id,doc_id,text,lemma
5108,468926,102125,Jammin' CRT.5,Jammin


In [98]:
mwe_df[mwe_df["text"] == "Traxxas Rustler XL-1 '09"]


,phrase_id,doc_id,text,lemma
6580,469441,102506,Traxxas Rustler XL-1 '09,Traxxas Rustler XL-1


In [99]:
mwe_df[mwe_df["text"] == "Żerań F S O"]


,phrase_id,doc_id,text,lemma
6916,350246,102673,Żerań F S O,Żerań FSO


In [100]:
mwe_df.loc[mwe_df["text"] == "Żerań F S O", "text"] = "Żerań FSO"

In [101]:
mwe_df.loc[mwe_df["text"] == "R C S Orzeł", "text"] = "RCS Orzeł"

In [102]:
mwe_df = mwe_df[mwe_df["text"] != "S K M"]
mwe_df = mwe_df[mwe_df["text"] != "R K S"]

In [103]:
mwe_df[mwe_df["lemma"].str.contains("ZTM")]

,phrase_id,doc_id,text,lemma
6897,352573,102669,Z T M,ZTM


In [104]:
mwe_df[mwe_df["lemma"].str.contains("WKD")]


,phrase_id,doc_id,text,lemma
6988,352545,102697,W K D,WKD


In [105]:
mwe_df[mwe_df["text"] == "átlátszó ("]

,phrase_id,doc_id,text,lemma
8097,481079,103921,átlátszó (,átlátszó


In [106]:
mwe_df[mwe_df["text"] == "Społecznej Rady Konsultacyjnej ds. aktualizacji „Strategii Rozwoju Bydgoszczy do 2015 roku”"]

,phrase_id,doc_id,text,lemma
5224,79818,102157,Społecznej Rady Konsultacyjnej ds. aktualizacji „Strategii Rozwoju Bydgoszczy do 2015 roku”,Społeczna Rada Konsultacyjna ds. aktualizacji„Strategii Rozwoju Bydgoszczy do 2015 roku


In [107]:
mwe_df.loc[mwe_df["text"] == 'Społecznej Rady Konsultacyjnej ds. aktualizacji „Strategii Rozwoju Bydgoszczy do 2015 roku”', "lemma"] = 'Społeczna Rada Konsultacyjna ds. aktualizacji „Strategii Rozwoju Bydgoszczy do 2015 roku"'

In [108]:
mwe_df[mwe_df["text"].str.contains("Fundacji Ośrodka")]

,phrase_id,doc_id,text,lemma
6812,71729,102602,"Fundacji Ośrodka "" Karta ""","Fundacja Ośrodka ""Karta """


In [109]:
mwe_df.loc[mwe_df["text"] == 'Fundacji Ośrodka " Karta "', ["text", "lemma"]] = ['Fundacji Ośrodka "Karta"', 'Fundacja Ośrodka "Karta"']

In [110]:
mwe_df[mwe_df["text"].str.contains("FILLER")] # error during creating dataset

,phrase_id,doc_id,text,lemma
6973,474289,102694,Dworca FILLER Centralnego,Dworzec Centralny
6978,350524,102696,Dworzec FILLER Śródmieście,Dworzec Śródmieście


In [111]:
mwe_df.loc[mwe_df["text"] == 'Dworca FILLER Centralnego', "text"] = 'Dworca Centralnego'
mwe_df.loc[mwe_df["text"] == 'Dworzec FILLER Śródmieście', "text"] = 'Dworzec Śródmieście'

In [112]:
mwe_df[mwe_df["text"].str.contains("Herburt")]

,phrase_id,doc_id,text,lemma
1202,50764,101263,Jan Herburt (Arłamowski,Jan Herburt (Arłamowski)


In [113]:
mwe_df[mwe_df["text"].str.contains("Naukę")]

,phrase_id,doc_id,text,lemma
4075,310468,101854,Naukę 2.0,Nauka2.0


In [114]:
mwe_df.loc[mwe_df["text"] == "Naukę 2.0", "lemma"] = "Nauka 2.0"


In [115]:
mwe_df[mwe_df["text"] == "Sesji Sejmu Dzieci i Młodzieży"]

,phrase_id,doc_id,text,lemma
3188,63764,101638,Sesji Sejmu Dzieci i Młodzieży,Sejm Dzieci i Młodzieży


In [116]:
mwe_df.loc[mwe_df["text"] == "Sesji Sejmu Dzieci i Młodzieży", "lemma"] = "Sesja Sejmu Dzieci i Młodzieży"

In [117]:
mwe_df[mwe_df["text"]=='ustawy o zmianie ustawy Kodeks postępowania karnego']

,phrase_id,doc_id,text,lemma
3279,63911,101644,ustawy o zmianie ustawy Kodeks postępowania karnego,Ustawao zmianie ustawy Kodeks postępowania karnego


In [118]:
mwe_df.loc[mwe_df["text"]=='ustawy o zmianie ustawy Kodeks postępowania karnego', "lemma"] = 'ustawa o zmianie ustawy Kodeks postępowania karnego'


In [119]:
mwe_df[mwe_df["text"].str.contains("o ujawnieniu pracy lub służby")]


,phrase_id,doc_id,text,lemma
4449,81044,101946,o zmianie ustawy o ujawnieniu pracy lub służby w organach bezpieczeństwa państwa lub współpracy z nimi w latach 1944–1990 osób pełniących funkcje publiczne,ustawa o ujawnieniu pracy lub służby w organach bezpieczeństwa państwa lub współpracy z nimi w latach 1944–1990 osób pełniących funkcje publiczne
4451,81046,101946,ustawie z dnia 11 kwietnia 1997 r. o ujawnieniu pracy lub służby w organach bezpieczeństwa państwa lub współpracy z nimi w latach 1944-1990 osób pełniących funkcje publiczne,Ustawa z dnia 11 kwietnia 1997 r. o ujawnieniu pracy lub służby w organach bezpieczeństwa państwa lub współpracy z nimi w latach 1944-1990 osób pełniących funkcje publiczne


In [120]:
mwe_df = mwe_df.drop(index=4449)

In [121]:
mwe_df.drop(mwe_df[mwe_df["text"].isin([
    "Dolce & Gabbana", # not a polish MWE
    "Lublin R-VIII a", # ???
    "BBC Radio 1's Live Lounge", # not a polish MWE
    "Jammin' CRT.5", # not a polish MWE
    "Traxxas Rustler XL-1 '09", # not a polish MWE
    "átlátszó (", # not a polish MWE
    "Z T M",
    "W K D",
    "Sly & the Family Stone",
    "modelem Jammin' X2"
])].index, inplace=True)

In [122]:
print(f"Shape of the Poleval 2019 dataset (after cleaning): {mwe_df.shape}")

Shape of the Poleval 2019 dataset (after cleaning): (3814, 4)


In [123]:
mwe_df.to_csv("mwe_cleaned_df.csv")

## Actual analysis

In [124]:
mwe_df = pd.read_csv("mwe_cleaned_df.csv", index_col=0)
print(f"Shape of the cleaned Poleval 2019 dataset: {mwe_df.shape}")

Shape of the cleaned Poleval 2019 dataset: (3814, 4)


In [125]:
mwe_df.head()

,phrase_id,doc_id,text,lemma
3,343872,99883,Ludwiga Mies van der Rohe,Ludwig Mies van der Rohe
7,343879,100499,samolotu PZL.23 „Karaś”,samolot PZL.23 „Karaś”
8,31843,100499,Ministerstwa Komunikacji,Ministerstwo Komunikacji
11,343880,100499,silnikiem Pratt-Whitney Wasp,silnik Pratt-Whitney Wasp
14,31839,100499,Stanisława Praussa,Stanisław Prauss


In [126]:
mwe_df.isna().sum()

phrase_id    0
doc_id       0
text         0
lemma        0
dtype: int64

In [144]:
def ignore(word):
    """
    Check if the word is a syntactically insignificant.
    Returns True if the word is a conjunction, preposition, particle, interjection,
    abbreviation, punctuation mark, or an unrecognized/ignored form.
    """
    analysis = morfeusz.analyse(word)
    ignore_tags = ('conj', 'prep', 'ign', 'brev', 'interp', 'part', 'qub')
    for _, _, interpretations in analysis:
        tag = interpretations[2]
        if tag.startswith(ignore_tags):
            return True
    return False

In [145]:
ignore("lub")

True

In [146]:
def analyze_pair(words, lemmas):
    """
    Analyze a 2 word expression.
    :param words: List of 2 words in the expression.
    :param lemmas: List of 2 lemmas corresponding to the words.
    :return: number, case, relation type and head of the expression.
    """
    # possible numbers and cases for the words
    number_case = {}
    part_of_speech = {}

    # returned values
    number = None
    case = None
    relation = None
    head = None

    #print(f"Analyzing pair: {words} with lemmas: {lemmas}")

    for i, word in enumerate(words):
        number_case_i = set()
        analyses = morfeusz.analyse(word)

        for _, _, interpretations in analyses:
            tag = interpretations[2].split(":")
            if len(tag) > 2:
                number_i = tag[1]
                case_i = tag[2].split('.')

                # Morfeusz can return more than 1 possibilty of case after the dot so I make a list of it 
                for c in case_i:
                    number_case_i.add((number_i, c))

                part_of_speech[word] = tag[0]

        number_case[i] = number_case_i

    available = [i for i in number_case if number_case[i]]

    # check if it is possible to find common number and case for the words in the mwe -> ZWIĄZEK ZGODY
    common_cases = set.intersection(*number_case.values())
    if len(common_cases)>=1:
        # if there is only one common pair(number, case), then it is an agreement relation and the number and case of the expression can be assigned to the whole expression
        
        if len(common_cases) == 1:
            number_case_head_list = list(common_cases)[0]
            number = number_case_head_list[0]
            case = number_case_head_list[1]
 
        else:
            number = [x[0] for x in list(common_cases)]
            case = [x[1] for x in list(common_cases)]
            
        relation = "agreement"

        # noun -> HEAD
        for i, (word, tag) in enumerate(part_of_speech.items()):
            if tag == 'subst':
                head = lemmas[i]
                break

        # if there is no noun in the expression, then the head is the first word in the expression
            else:
                head = lemmas[0]

    elif [w_i for w_i, l_i in zip(words, lemmas) if w_i == l_i]:
        # government relation    
        for i, word in enumerate(words):
            if word == lemmas[i]:
                continue
            else: 
                if number_case[i]:
                    head = lemmas[i]
                    number_case_head_list = list(number_case[i])
                    number = [x[0] for x in number_case_head_list]
                    case = [x[1] for x in number_case_head_list]
                    relation = "government"
        
    if len(available) == 1:
        #print(f"Only one word has a valid analysis: {words[available[0]]} with lemma {lemmas[available[0]]}, words: {words}, lemmas: {lemmas}, number_case: {number_case}")
        # if there is only one word in the expression that has a valid analysis, then it is the head of the expression
        head = lemmas[available[0]]
        number_case_head_list = list(number_case[available[0]])
        number = [x[0] for x in number_case_head_list]
        case = [x[1] for x in number_case_head_list]
        
        idx_j = 1 - available[0]  # index of the second word in the pair
        if words[idx_j] == lemmas[idx_j]:
            # if the second word is the same as the lemma, then it is a government relation
            relation = "government"
        else:
            relation = "agreement"

    return number, case, relation, head


def analyze_mwe(phrase, lemma):
    """
    Analyze a multi-word expression.
    :param phrase: The multi-word expression.
    :param lemma: The lemma of the multi-word expression.
    :return: analysis results - possible: numbers, cases, relations, heads, pairs.
    """

    phrase = phrase.replace(" - ", "-")
    phrase = phrase.replace("___", "")

    
    words = phrase.strip().split()
    lemmas = lemma.strip().split() 

    numbers = []
    cases = []
    relations = []
    heads = []
    pairs = []

    print(f"Analyzing phrase: {phrase} with lemma: {lemma}")

    i = 0
    while i < len(words)-1:
        if ignore(words[i+1]) and i + 2 < len(words):
            pair_words = [words[i], words[i+2]]
            pair_lemmas = [lemmas[i], lemmas[i+2]]
            i += 2 # skip word that is ignored
        
        else:
            pair_words = words[i:i+2]
            pair_lemmas = lemmas[i:i+2]
            i += 1 

        # analyze the pair of words
        analyses = analyze_pair(pair_words, pair_lemmas)
        number, case, relation, head = analyses
        if type(number) is list and len(number) == 1:
            number = number[0]
        if type(case) is list and len(case) == 1:
            case = case[0]
        pairs.append(tuple(pair_words))
        numbers.append(number)  
        cases.append(case)
        relations.append(relation)
        heads.append(head)


    return numbers, cases, relations, heads, pairs

In [147]:
mwe_df[["numbers", "cases", "relations", "heads", "pairs"]] = mwe_df.apply(lambda row: pd.Series(analyze_mwe(row["text"], row["lemma"])), axis=1)

Analyzing phrase: Ludwiga Mies van der Rohe with lemma: Ludwig Mies van der Rohe
Analyzing phrase: samolotu PZL.23 „Karaś” with lemma: samolot PZL.23 „Karaś”
Analyzing phrase: Ministerstwa Komunikacji with lemma: Ministerstwo Komunikacji
Analyzing phrase: silnikiem Pratt-Whitney Wasp with lemma: silnik Pratt-Whitney Wasp
Analyzing phrase: Stanisława Praussa with lemma: Stanisław Prauss
Analyzing phrase: Państwowych Zakładach Lotniczych w Warszawie with lemma: Państwowe Zakłady Lotnicze w Warszawie
Analyzing phrase: Państwowych Zakładach Lotniczych with lemma: Państwowe Zakłady Lotnicze
Analyzing phrase: Trzeciego Świata with lemma: Trzeci Świat
Analyzing phrase: południowej Jutlandii with lemma: południowa Jutlandia
Analyzing phrase: województwie opolskim with lemma: województwo opolskie
Analyzing phrase: powiecie kluczborskim with lemma: powiat kluczborski
Analyzing phrase: Carla Gotharda Langhansa with lemma: Carl Gothard Langhans
Analyzing phrase: gminie Kluczbork with lemma: gmina 

In [148]:
mwe_df.sample(20)

,phrase_id,doc_id,text,lemma,numbers,cases,relations,heads,pairs
7740,484183,103807,tematu czasu teraźniejszego,temat czasu teraźniejszego,"[sg, sg]","[gen, gen]","[agreement, agreement]","[temat, czasu]","[(tematu, czasu), (czasu, teraźniejszego)]"
7702,484106,103797,ogólnej teorii względności,ogólna teoria względności,"[[sg, sg, sg], [sg, sg, sg, pl]]","[[loc, dat, gen], [loc, dat, gen, gen]]","[agreement, agreement]","[teoria, teoria]","[(ogólnej, teorii), (teorii, względności)]"
1509,53204,101309,3 Pułku Bersalierów,3 Pułk Bersalierów,"[[sg, sg, sg], [sg, sg, sg]]","[[loc, gen, voc], [loc, gen, voc]]","[government, government]","[Pułk, Pułk]","[(3, Pułku), (Pułku, Bersalierów)]"
6578,469439,102505,śrubami AXA088,śruby AXA088,[pl],[inst],[government],[śruby],"[(śrubami, AXA088)]"
6005,338137,102383,Jarosława Fajfera,Jarosław Fajfer,"[[sg, sg]]","[[acc, gen]]",[agreement],[Jarosław],"[(Jarosława, Fajfera)]"
6711,466396,102558,Ministra Pracy i Polityki Społecznej,Minister Pracy i Polityki Społecznej,"[sg, sg, sg]","[gen, gen, gen]","[agreement, agreement, agreement]","[Minister, Pracy, Polityki]","[(Ministra, Pracy), (Pracy, Polityki), (Polityki, Społecznej)]"
4559,76819,101965,Władcy Pierścieni,Władca Pierścieni,"[[sg, pl, pl, sg, sg]]","[[gen, nom, voc, loc, dat]]",[government],[Władca],"[(Władcy, Pierścieni)]"
9125,329390,107360,Ruchu Chorzów,Ruch Chorzów,"[[sg, sg, sg, sg]]","[[loc, dat, gen, voc]]",[government],[Ruch],"[(Ruchu, Chorzów)]"
3881,473145,101799,Stowarzyszenia Dziennikarzy Polskich,Stowarzyszenie Dziennikarzy Polskich,"[pl, [pl, pl]]","[acc, [gen, acc]]","[agreement, agreement]","[Stowarzyszenie, Dziennikarzy]","[(Stowarzyszenia, Dziennikarzy), (Dziennikarzy, Polskich)]"
3011,466691,101582,gazu skroplonego,gaz skroplony,[sg],[gen],[agreement],[gaz],"[(gazu, skroplonego)]"


In [143]:
analyze_pair("Dzieciątka Jezus", "Dzieciątka Jezus")

(None, None, None, None)

In [136]:
analyze_mwe("kawalerem Legii Honorowej", "kawaler Legii Honorowej")

Analyzing phrase: kawalerem Legii Honorowej with lemma: kawaler Legii Honorowej


([['sg'], ['sg', 'sg', 'sg']],
 [['inst'], ['loc', 'dat', 'gen']],
 ['government', 'agreement'],
 ['kawaler', 'Legii'],
 [('kawalerem', 'Legii'), ('Legii', 'Honorowej')])

In [134]:
analyze_pair(["Trzech", "Króli"], ["Trzech", "Króli"])	

('pl', 'gen', 'agreement', 'Króli')

In [ ]:
analyze_mwe("Kazania na święto Trzech Króli", "Kazanie na święto Trzech Króli")

Analyzing phrase: Kazania na święto Trzech Króli with lemma: Kazanie na święto Trzech Króli


([['pl', 'pl', 'sg', 'pl'], None, 'pl'],
 [['nom', 'voc', 'gen', 'acc'], None, 'gen'],
 ['government', None, 'agreement'],
 ['Kazanie', None, 'Króli'],
 [('Kazania', 'święto'), ('święto', 'Trzech'), ('Trzech', 'Króli')])

In [57]:
morfeusz.analyse("Kazania na święto Trzech Króli")

[(0,
  1,
  ('Kazania', 'kazanie', 'subst:sg:gen:n:ncol', ['nazwa_pospolita'], [])),
 (0,
  1,
  ('Kazania',
   'kazanie',
   'subst:pl:nom.acc.voc:n:ncol',
   ['nazwa_pospolita'],
   [])),
 (0,
  1,
  ('Kazania', 'Kazanie', 'subst:sg:gen:n:ncol', ['nazwa_geograficzna'], [])),
 (0,
  1,
  ('Kazania',
   'Kazanie',
   'subst:pl:nom.acc.voc:n:ncol',
   ['nazwa_geograficzna'],
   [])),
 (0, 1, ('Kazania', 'Kazań', 'subst:sg:gen:m3', ['nazwa_geograficzna'], [])),
 (0, 1, ('Kazania', 'kazać', 'ger:pl:nom.acc:n:imperf.perf:aff', [], [])),
 (0, 1, ('Kazania', 'kazać', 'ger:sg:gen:n:imperf.perf:aff', [], [])),
 (1, 2, ('na', 'na:I', 'interj', [], [])),
 (1, 2, ('na', 'na:P', 'prep:acc', [], [])),
 (1, 2, ('na', 'na:P', 'prep:loc', [], [])),
 (2,
  3,
  ('święto',
   'święto:S',
   'subst:sg:nom.acc.voc:n:ncol',
   ['nazwa_pospolita'],
   [])),
 (2, 3, ('święto', 'święto:D', 'adv:pos', [], ['daw.'])),
 (3, 4, ('Trzech', 'trzy', 'num:pl:acc:m1:rec:ncol', [], [])),
 (3, 4, ('Trzech', 'trzy', 'num